In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

## Matchdata Reading from CSV File

In [ ]:
matchdata=pd.read_csv('https://raw.githubusercontent.com/prakharmohan4/BYOP/main/IPL%20Ball-by-Ball%202008-2020.csv')

In [ ]:
matchdata.head()

## Aggregating Batsmen Data

In [ ]:
        batsmen_score = pd.DataFrame(matchdata.groupby(['id', 'batsman'])['batsman_runs'].sum()).\
            rename(columns={"batsman_runs": "total_runs"})
        batsmen_scores_30 = pd.DataFrame(((matchdata.groupby(['id', 'batsman'])['batsman_runs'].sum())>=30).astype(int)).\
            rename(columns={"batsman_runs": "total_runs_30"})
        batsmen_scores_50 = pd.DataFrame(((matchdata.groupby(['id', 'batsman'])['batsman_runs'].sum())>=50).astype(int)).\
            rename(columns={"batsman_runs": "total_runs_50"})
        batsmen_scores_100 = pd.DataFrame(((matchdata.groupby(['id', 'batsman'])['batsman_runs'].sum())>=100).astype(int)).\
            rename(columns={"batsman_runs": "total_runs_100"})
        batsmen_scores_0 = pd.DataFrame(((matchdata.groupby(['id', 'batsman'])['batsman_runs'].sum())==0).astype(int)).\
            rename(columns={"batsman_runs": "total_runs_0"})
        batsmen_ball_faced = pd.DataFrame(matchdata.groupby(['id', 'batsman'])['over'].count()).\
            rename(columns={"over": "total_balls_faced"})
        batsmen_ball_faced_legal = pd.DataFrame(matchdata[matchdata['ball'] <=6].groupby(['id', 'batsman'])['ball'].count()).\
            rename(columns={"ball": "total_legal_balls_faced"})
        batsmen_scores6 = pd.DataFrame(matchdata[matchdata['batsman_runs'] == 6].groupby(['id', 'batsman'])['batsman_runs'].count()).\
            rename(columns={"batsman_runs": "run_6"})
        batsmen_scores4 = pd.DataFrame(matchdata[matchdata['batsman_runs'] == 4].groupby(['id', 'batsman'])['batsman_runs'].count()).\
            rename(columns={"batsman_runs": "run_4"})
        batsmen_position = pd.DataFrame(matchdata.groupby(['id', 'batsman'])['is_wicket'].min())
       
        
        df_list = [batsmen_score, batsmen_ball_faced, batsmen_ball_faced_legal, batsmen_scores6, batsmen_scores4, batsmen_position,batsmen_scores_30,batsmen_scores_50,batsmen_scores_100,batsmen_scores_0]
        batsmen_summary = pd.concat(df_list, join='outer', axis=1).fillna(np.nan).reset_index()
        batsmen_summary = pd.merge(batsmen_summary, matchdata[['id', 'batsman', 'inning', 'batting_team', 'bowling_team']].
                                   drop_duplicates(), on=['id', 'batsman'], how='left')
        batsmen_summary.rename(columns={'inning': 'batsmen_innings', 'batsman': 'playername', 'batting_team': 'PlayingTeam',
                                        'bowling_team': 'Opponent Team'}, inplace=True)

In [ ]:
batsmen_summary

## Aggregating Bowler Data

In [ ]:
        bowler_wickets = pd.DataFrame(matchdata[((matchdata['is_wicket'] == 1) & (~matchdata['dismissal_kind'].isin(['run out','retired hurt'])))].
                                      groupby(['id', 'bowler'])['is_wicket'].count()).\
            rename(columns={"is_wicket": "total_wickets"})
        bowler_overs_bow = pd.DataFrame(matchdata.groupby(['id', 'bowler'])['over'].count()).\
            rename(columns={"over": "total_balls_bowled"})
        bowler_ball_faced_legal = pd.DataFrame(matchdata[matchdata['ball'] <=6].groupby(['id', 'bowler'])['ball'].count()).\
            rename(columns={"ball": "total_legal_balls_bowled"})
        # calculating the number of maiden overs bowled in a match
        bowler_runs_given = pd.DataFrame(matchdata.groupby(['id', 'bowler'])['batsman_runs'].sum()).\
            rename(columns={"batsman_runs": "total_runs_given"})
        bowler_wickets_3 = pd.DataFrame((matchdata[((matchdata['is_wicket'] == 1) & (~matchdata['dismissal_kind'].isin(['run out','retired hurt'])))].
                                      groupby(['id', 'bowler'])['is_wicket'].count())==3).astype(int).\
            rename(columns={"is_wicket": "total_wickets_3"})
        bowler_wickets_4 = pd.DataFrame((matchdata[((matchdata['is_wicket'] == 1) & (~matchdata['dismissal_kind'].isin(['run out','retired hurt'])))].
                                      groupby(['id', 'bowler'])['is_wicket'].count())==4).astype(int).\
            rename(columns={"is_wicket": "total_wickets_4"})
        bowler_wickets_5 = pd.DataFrame((matchdata[((matchdata['is_wicket'] == 1) & (~matchdata['dismissal_kind'].isin(['run out','retired hurt'])))].
                                      groupby(['id', 'bowler'])['is_wicket'].count())==5).astype(int).\
            rename(columns={"is_wicket": "total_wickets_5"})
        bowler_maiden = pd.DataFrame(matchdata.groupby(['id', 'bowler', 'over'])['total_runs'].sum()).reset_index()
        bowler_maiden['maiden_overs'] = np.where(bowler_maiden['total_runs'] == 0, 1, 0)
        bowler_maiden = bowler_maiden.groupby(['id', 'bowler'])['maiden_overs'].sum()
        
        #legbyes to be considered
        
        df_list = [bowler_wickets, bowler_overs_bow, bowler_ball_faced_legal, bowler_runs_given,bowler_wickets_3,bowler_wickets_4,bowler_wickets_5,bowler_maiden]
        bowler_summary = pd.concat(df_list, join='outer', axis=1).fillna(np.nan).reset_index()
        # adding additional columns just in case we need for modeling
        bowler_summary = pd.merge(bowler_summary, matchdata[['id', 'bowler', 'inning', 'batting_team', 'bowling_team']].drop_duplicates(), on=['id', 'bowler'], how='left')
        bowler_summary.rename(columns={'inning': 'bowlers_innings', 'bowler': 'playername', 'batting_team': 'Playing team',
                                       'bowling_team': 'Opponent Team'}, inplace=True)
        bowler_summary['economy_rate'] = (bowler_summary['total_runs_given'] * 6) / bowler_summary['total_legal_balls_bowled']
        bowler_summary['eco less than 5'] = (bowler_summary['economy_rate']<=5).astype(int)
        bowler_summary['eco 5-5.99'] = ((bowler_summary['economy_rate']<6)&(bowler_summary['economy_rate']>=5)).astype(int)
        bowler_summary['eco 6-7'] = ((bowler_summary['economy_rate']>=6)&(bowler_summary['economy_rate']<7)).astype(int)
        bowler_summary['eco 7-10'] = ((bowler_summary['economy_rate']>=7)&(bowler_summary['economy_rate']<10)).astype(int)
        bowler_summary['eco 11-12'] = ((bowler_summary['economy_rate']>=11)&(bowler_summary['economy_rate']<12)).astype(int)
        bowler_summary['eco greater than 12'] = (bowler_summary['economy_rate']>12).astype(int)

In [ ]:
bowler_summary

## Merging Bowler and Batsmen Stats

In [ ]:
ipl_merged_scorecard = pd.merge(batsmen_summary, bowler_summary, on=['id', 'playername'], how='outer')

In [ ]:
ipl_merged_scorecard

## Calculating Player Role

In [ ]:
#All Rounder - If the Player has faced atleast 8 balls and bowled atleast 6 balls then he's an All rounder
#Bowler - If the Player has bowled more than 6 balls then he's a bowler 
# Batsem- If the Player has faced more than 8 balls then he's a batsmen (deafault) 

MINAVGBALLSFACED = 8
MINAVGBOWLSBOWLED = 6
player_avg = ipl_merged_scorecard[['playername', 'total_balls_faced', 'total_balls_bowled']].fillna(0)
player_avg = pd.DataFrame(player_avg.groupby('playername')[['total_balls_faced', 'total_balls_bowled']].mean())
conditions = [((player_avg['total_balls_faced'] >= MINAVGBALLSFACED) & (player_avg['total_balls_bowled'] >= MINAVGBOWLSBOWLED)),
              (player_avg['total_balls_bowled'] >= MINAVGBOWLSBOWLED)]
choices = ['AllRounder', 'Bowler']
player_avg['playing_role'] = np.select(conditions, choices, default='Batsmen')
player_avg = player_avg.reset_index()

In [ ]:
ipl_merged_scorecard = ipl_merged_scorecard.merge(player_avg[["playername","playing_role"]], how="inner",on="playername")


In [ ]:
ipl_merged_scorecard

## Points Calculation

In [ ]:
pointsconfig = {
        'total_runs': 1,
        'run_6': 2,
        'run_4': 1,
        '>=30':4,
        '>=50': 8,
        '>=100': 16,
        'duck': -2,
        'total_wickets': 25,
        '<=3W':4,
        '>=4W': 8,
        '>=5W': 16,
        'maiden_overs': 8,
        '<=5E': 6,
        '<6E': 4,
        '<7E': 2,
        '>10E': -2,
        '>11E': -4,
        '>12E': -6
    }

### Batsmen Points

In [ ]:
        ipl_merged_scorecard['total_runs_points'] = ipl_merged_scorecard['total_runs'] * pointsconfig['total_runs']
        ipl_merged_scorecard['run_6_points'] = pointsconfig['run_6'] *  ipl_merged_scorecard['run_6']
        ipl_merged_scorecard['run_4_points'] = pointsconfig['run_4'] *  ipl_merged_scorecard['run_4']
        ipl_merged_scorecard['30_Run_Points'] = pointsconfig['>=30'] *  ipl_merged_scorecard['total_runs_30']
        ipl_merged_scorecard['50_Run_Points'] = pointsconfig['>=50'] *  ipl_merged_scorecard['total_runs_50']
        ipl_merged_scorecard['100_Run_Points'] = pointsconfig['>=100'] *  ipl_merged_scorecard['total_runs_100']
        ipl_merged_scorecard['duck_Points'] = pointsconfig['duck'] *  ipl_merged_scorecard['total_runs_0']
        ipl_merged_scorecard['total_bat_points'] = np.nan
        ipl_merged_scorecard['total_bat_points'] = ipl_merged_scorecard['total_runs_points'].add(ipl_merged_scorecard['run_6_points'], fill_value=0). \
            add(ipl_merged_scorecard['run_4_points'], fill_value=0).add(ipl_merged_scorecard['30_Run_Points'], fill_value=0).\
            add(ipl_merged_scorecard['50_Run_Points'], fill_value=0).add(ipl_merged_scorecard['100_Run_Points'], fill_value=0).\
            add(ipl_merged_scorecard['duck_Points'], fill_value=0)
        ipl_merged_scorecard['total_bat_points'] = np.where(ipl_merged_scorecard['total_balls_faced'] >= 1, ipl_merged_scorecard['total_bat_points'], np.nan)

### Bowler Points

In [ ]:
        ipl_merged_scorecard['total_wicket_points'] = ipl_merged_scorecard['total_wickets'] * pointsconfig['total_wickets']
        ipl_merged_scorecard['<=5E_points'] = pointsconfig['<=5E'] *  ipl_merged_scorecard['eco less than 5']
        ipl_merged_scorecard['<6E_points'] = pointsconfig['<6E'] *  ipl_merged_scorecard['eco 5-5.99']
        ipl_merged_scorecard['<7E_points'] = pointsconfig['<7E'] *  ipl_merged_scorecard['eco 6-7']
        ipl_merged_scorecard['>10E_points'] = pointsconfig['>10E'] *  ipl_merged_scorecard['eco 7-10']
        ipl_merged_scorecard['>11E_points'] = pointsconfig['>11E'] *  ipl_merged_scorecard['eco 11-12']
        ipl_merged_scorecard['>12E_points'] = pointsconfig['>12E'] *  ipl_merged_scorecard['eco greater than 12']
        ipl_merged_scorecard['3W_points'] = pointsconfig['>=100'] *  ipl_merged_scorecard['total_wickets_3']
        ipl_merged_scorecard['4W_points'] = pointsconfig['>=100'] *  ipl_merged_scorecard['total_wickets_4']
        ipl_merged_scorecard['5W_points'] = pointsconfig['>=100'] *  ipl_merged_scorecard['total_wickets_5']
        ipl_merged_scorecard['maiden_overs_points'] = pointsconfig['>=100'] *  ipl_merged_scorecard['maiden_overs']
        ipl_merged_scorecard['total_bowl_points'] = np.nan
        ipl_merged_scorecard['total_bowl_points'] = ipl_merged_scorecard['total_wicket_points'].add(ipl_merged_scorecard['<=5E_points'], fill_value=0). \
            add(ipl_merged_scorecard['<6E_points'], fill_value=0).add(ipl_merged_scorecard['<7E_points'], fill_value=0).\
            add(ipl_merged_scorecard['>10E_points'], fill_value=0).add(ipl_merged_scorecard['>11E_points'], fill_value=0).\
            add(ipl_merged_scorecard['>12E_points'], fill_value=0).add(ipl_merged_scorecard['3W_points'], fill_value=0).\
            add(ipl_merged_scorecard['4W_points'], fill_value=0).add(ipl_merged_scorecard['5W_points'], fill_value=0).\
            add(ipl_merged_scorecard['maiden_overs_points'], fill_value=0)
        ipl_merged_scorecard['total_bowl_points'] = np.where(ipl_merged_scorecard['total_balls_bowled'] >= 1, ipl_merged_scorecard['total_bowl_points'], np.nan)

In [ ]:
ipl_merged_scorecard.info()

### Merging Bowler and Batsmen Points

In [ ]:
ipl_merged_scorecard['total_points'] = ipl_merged_scorecard['total_bat_points'].add(ipl_merged_scorecard['total_bowl_points'], fill_value=0)

### Dropping Irrelevant Columns

In [ ]:
ipl_merged_scorecard.drop(['id','PlayingTeam','Opponent Team_x','Playing team','Opponent Team_y'],axis=1,inplace=True)

In [ ]:
ipl_merged_scorecard.columns

In [ ]:
#ipl_merged_scorecard.replace([np.inf, -np.inf], np.nan, inplace=True)
ipl_merged_scorecard.fillna(0,inplace=True)

### One hot Encoding for playing_role

In [ ]:
one_hot = pd.get_dummies(ipl_merged_scorecard['playing_role'],drop_first=True)
 #Drop column as it is now encoded
ipl_merged_scorecard = ipl_merged_scorecard.drop('playing_role',axis = 1)
#Join the encoded df


In [ ]:
ipl_merged_scorecard = ipl_merged_scorecard.join(one_hot)
ipl_merged_scorecard

### Rearranging Columns

In [ ]:
ipl_merged_scorecard= ipl_merged_scorecard[['playername','total_runs', 'total_balls_faced', 'total_legal_balls_faced', 'run_6',
       'run_4', 'is_wicket', 'total_runs_30', 'total_runs_50',
       'total_runs_100', 'total_runs_0', 'batsmen_innings', 'total_wickets',
       'total_balls_bowled', 'total_legal_balls_bowled', 'total_runs_given',
       'total_wickets_3', 'total_wickets_4', 'total_wickets_5', 'maiden_overs',
       'bowlers_innings', 'economy_rate', 'eco less than 5', 'eco 5-5.99',
       'eco 6-7', 'eco 7-10', 'eco 11-12', 'eco greater than 12',
       'Bowler', 'Batsmen','total_points']]

In [ ]:
df1=ipl_merged_scorecard.copy()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
ipl_merged_scorecard.playername.value_counts().sort_values(ascending=False)

In [ ]:
ipl_merged_scorecard

### Removing players who have played less than 5 matches

In [ ]:
df = ipl_merged_scorecard.groupby('playername').filter(lambda x : len(x)>5)
df

## LR Model - tried it with a specific player name

In [ ]:
#df['playername'].unique()

In [ ]:
#
#X_train, X_test, y_train, y_test=train_test_split(X,y, random_state = 9999)
#lr = LinearRegression()
#lr.fit(X_train,y_train)
#lr.coef_
#lr=linear_model.LinearRegression().fit(X_train,y_train)
#lr["B Akhil"]=lr

In [ ]:
##Checkin the shape -- SHAPE ERROR - Size of X and y are different
#X.shape, y.shape

## POP FUNCTION LR MODEL

In [ ]:
# New Code
models={}
for i in df['playername']:
    X = df[df['playername']==i].drop(columns=['total_points','playername'])
    y=df[df['playername']==i]['total_points']
    X_train, X_test, y_train, y_test=train_test_split(X,y, random_state = 42,  test_size=0.25)
    lr = LinearRegression().fit(X_train,y_train)
    models[i]=lr  

In [ ]:
lr={}
for i in df['playername']:
    X = df[df['playername']==i].pop ('total_points')
    print(X)
    y=df[df['playername']==i]['total_points'] # Should we add the pop function here?
    print(y)
    X_train, X_test, y_train, y_test=train_test_split(X,y, random_state = 9999)
    print(X_train)
    print(y_train)
    print(X_test)
    print(y_test)
    lr = linear_model.LinearRegression().fit(X_train,y_train)
    lr[i]=lr  
    

## Fitting LR Model and checking errors

In [ ]:
## Fitting LR model
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
lr_dict={}
errors = { }
for i in df['playername'].unique():
    X = df[df['playername']==i].iloc [ : , 1:-1]
    y=df[df['playername']==i]['total_points']
    X_train, X_test, y_train, y_test=train_test_split(X,y, random_state = 9999)
    lr = linear_model.LinearRegression()
    lr.fit(X_train,y_train)
    y_pred = lr.predict(X_test)
    lr_dict[i]=[np.array(y_test), y_pred]  
    errors[i] = { "R2 Score" : round(r2_score(y_test , y_pred), 2),
                "RMSE" : round(np.sqrt(mean_squared_error(y_test , y_pred)),2), 
                 "MSE": round(mean_squared_error(y_test , y_pred),2), 
                 "MAE": round(mean_absolute_error(y_test , y_pred),2)} 


In [ ]:
#Printing Actual values and Predicted Values
print(lr_dict["B Akhil"])
# Actual values , Pred Values

In [ ]:
# Printing Errors for the player
print(errors["B Akhil"])

## Ridge Regresion

In [ ]:
for i in df['playername']:
    player = df[df['playername']==i]
    player_new = player.dropna()

In [ ]:
player_new

In [ ]:
# Using ridge regression to predict the next match's performance based on the same player's performance in past

models = pd.DataFrame()

for i in df['playername']:
    player = df[df['playername']==i]
    player_new = player.dropna()

    X = player_new[player_new.columns[1:11]]
    y = player_new[player_new.columns[31:32]]

    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.10,random_state=5)

    ridge = pd.DataFrame()

    for j in range(0,101):
        points = linear_model.Ridge(alpha = j).fit(X_train,y_train)
        ridge_df = pd.DataFrame({'Alpha':pd.Series(j),'Train':pd.Series(points.score(X_train,y_train)),'Test':pd.Series(points.score(X_test,y_test))})
        ridge = ridge.append(ridge_df)

    ridge['Average'] = ridge[['Train', 'Test']].mean(axis=1)


    try:
        k = ridge[ridge[ 'Average' ]==ridge[ 'Average' ].max()][ 'Alpha' ][0]
        k = k.head(1)[0]

    except:

        k=ridge[ridge[ 'Average' ]==ridge['Average'].max()][ 'Alpha' ][0]

    next_runs = linear_model.Ridge(alpha = k)
    next_runs.fit(X_train,y_train)

    X = player_new[player_new.columns[1:11]]
    y = player_new[player_new.columns[32:33]]

    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.10,random_state=5)

    ridge = pd.DataFrame()

    for j in range(0,101):
        points = linear_model.Ridge(alpha = j).fit(X_train,y_train)
        ridge_df = pd.DataFrame({'Alpha':pd.Series(j),'Train':pd.Series(points.score(X_train,y_train)),'Test':pd.Series(points.score(X_test,y_test))})
        ridge = ridge.append(ridge_df)

    ridge['Average'] = ridge[['Train', 'Test']].mean(axis=1)

    try:
        k = ridge[ridge[ 'Average' ]==ridge[ 'Average' ].max()][ 'Alpha'][0]
        k = k.head(1)[0]

    except:
        k = ridge[ridge[ 'Average' ]==ridge['Average'].max()][ 'Alpha' ][0]

    next_balls = linear_model.Ridge(alpha = k)
    next_balls.fit(X_train, y_train)

    X = player_new[player_new.columns[12:27]]
    y = player_new[player_new.columns[34:35]]

    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.10,random_state=5)

    ridge = pd.DateFrame()

    for j in range(0,101):
        points = linear_model.Ridge(alpha = j).fit(X_train,y_train)
        ridge_df = pd.DataFrame({'Alpha':pd.Series(j), 'Train' :pd.Series(points.score(X_train,y_train)),'Test':pd.Series(points.score(X_test,y_test))})
        ridge = ridge.append(ridge_af)

    ridge['Average'] = ridge[['Train','Test']].mean(axis=1)

    try:
        k = ridge[ridge[ 'Average' ]==ridge[ 'Average' ].max()][ 'Alpha' ][0]
        k = k.head(1)[0]
                                                            
    except:
        k = ridge[ridge[ 'Average' ]==ridge['Average'].max()][ 'Alpha' ][0]

    nex_wkts = linear_model.Ridge(alpha = k)
    nex_wkts.fit(X_train, y_train)
  
    X = player_new[player_new.columns[14:27]]
    y = player_new[player_new.columns[33:34]]

    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.10,random_state=5)

    ridge = pd.DataFrame()

    for j in range(0,101):
        points = linear_model.Ridge(alpha = j).fit(X_train,y_train)
        ridge_df = pd.DataFrame({'Alpha':pd.Series(j),'Train' :pd.Series(points.score(X_train,y_train)),'Test':pd.Series(points.score(X_test,y_test))})
        ridge = ridge.append(ridge_df)

    ridge['Average'] = ridge[['Train', 'Test']].mean(axis=1)

    try:
        k = ridge[ridge[ 'Average' ]==ridge['Average'].max()]['Alpha'][0]
        k = k.head(1)[0]
    except:
        k = ridge[ridge[ 'Average' ]==ridge[ 'Average' ].max()]['Alpha'][0]

    next_overs = linear_model.Ridge(alpha = k)
    next_overs.fit(X_train, y_train)

    X = player_new[player_new.columns[14:27]]
    y = player_new[player_new.columns[33:34]]

    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.10,random_state=5)

    ridge = pd.DataFrame()

    for j in range(0,101):
        points = linear_model.Ridge(alpha = j).fit(X_train,y_train)
        ridge_df = pd.DataFrame({'Alpha':pd.Series(j),'Train':pd.Series(points.score(X_train,y_train)), 'Test':pd.Series(points.score(X_test,y_test))})
        ridge = ridge.append(ridge_af)

    ridge['Average'] = ridge[['Train', 'Test']].mean(axis=1)

    try:
        k = ridge[ridge[ 'Average' ]==ridge['Average'].max()]['Alpha'][0]
        k = k.head(1)[0]

    except:
        k = ridge[ridge[ 'Average' ]==ridge['Average'].max()][ 'Alpha' ][0]

    next_runs_given = linear_model.Ridge(alpha = k)
    next_runs_given.fit(X_train, y_train)

    latest = player.groupby('playername').tail(1)
    latest['next_runs'] = next_runs.predict(latest[latest.columns[1:13]])
    latest['next_balls'] = next_balls.predict(latest[latest.columns[1:13]])
    latest['next_overs'] = next_overs.predict(latest[latest.columns[14:27]])
    latest['next_runs_given'] = next_runs_given.predict(latest[latest.columns[14:27]])
    latest['nex_wkts'] = nex_wkts.predict(latest[latest.columns[14:27]])
    models = models.append(latest)

models['next_runs_given'] = round(models[ 'next_runs_given'],0)

models['next_overs'] = round(models['next_overs'],0)

models['next_runs'] = round(models['next_runs'],0)

models['next_balls'] = round(models['next_balls'],0)

models['nex_wkts'] = round(models[ 'nex_wkts'],0)

models
